In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.animation import FuncAnimation
from geometry_msgs.msg import Pose, Twist
import rospy
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose
from math import pow, atan2, sqrt


# Tank-drive dynamics model

\begin{equation}
u=[v\ \omega]'\\
z=[x\ y\ \theta]'\\
|v|\leq v_{max}, |\omega|\leq \omega_{max}
\end{equation}

$\theta$ increases counter-clockwise.

\begin{equation}
\dot{z}=[v\cos{\theta}\ v\sin{\theta}\ \omega]'=[u_1\cos{z_3} \ u_1\sin{z_3} \ u_2]'\ (0)
\end{equation}

We anticipate the equation (0) is used by turtlesim and Gazebo Turtlebot3 to implement tank-drive update.

Furthermore, we anticipate the exact update formula used by the two simulation package above to be something similar to the formula below: 

for small $\Delta t$
\begin{equation}
z(t+\Delta t) \approx z(t)+ [v\cos{\theta}\Delta t\ v\sin{\theta}\Delta t\ \omega\Delta t]'=z(t)+ [u_1(t)\cos{z_3(t)} \ u_1(t)\sin{z_3(t)} \ u_2(t)]' \Delta t\ \ \ \ \ (1)
\end{equation}

# Construct Controllers for turtlesim simulator

The following code is adopted and modified from http://wiki.ros.org/turtlesim/Tutorials/Go%20to%20Goal

In [22]:
# The default turtlebot speed is SUUUUPER slow.
# BURGER_MAX_LIN_VEL = 0.22
# BURGER_MAX_ANG_VEL = 2.84

BURGER_MAX_LIN_VEL = 0.22*10
BURGER_MAX_ANG_VEL = 2.84*10
turtlebot3_model='burger'

def constrain(input, low, high):
        if input < low:
          input = low
        elif input > high:
          input = high
        else:
          input = input
        return input

class TurtleBot:

    def __init__(self):
        # Creates a node with name 'turtlebot_controller' and make sure it is a
        # unique node (using anonymous=True).
        rospy.init_node('turtlebot_controller', anonymous=True)

        # Publisher which will publish to the topic '/turtle1/cmd_vel'.
        self.velocity_publisher = rospy.Publisher('/turtle1/cmd_vel',
                                                  Twist, queue_size=10)

        # A subscriber to the topic '/turtle1/pose'. self.update_pose is called
        # when a message of type Pose is received.
        self.pose_subscriber = rospy.Subscriber('/turtle1/pose',
                                                Pose, self.update_pose)

        self.pose = None
        self.rate = rospy.Rate(10)
        

    def update_pose(self, data):
        """Callback function which is called when a new message of type Pose is
        received by the subscriber."""
        self.pose = data
        self.pose.x = round(self.pose.x, 4)
        self.pose.y = round(self.pose.y, 4)

    def euclidean_distance(self, goal_pose):
        if self.pose is None:
            return np.inf
        """Euclidean distance between current pose and the goal."""
        return sqrt(pow((goal_pose.x - self.pose.x), 2) +
                    pow((goal_pose.y - self.pose.y), 2))
    def legal_linear_vel(self, v):
        if turtlebot3_model == "burger":
          v = constrain(v, -BURGER_MAX_LIN_VEL, BURGER_MAX_LIN_VEL)
        elif turtlebot3_model == "waffle" or turtlebot3_model == "waffle_pi":
          v = constrain(v, -WAFFLE_MAX_LIN_VEL, WAFFLE_MAX_LIN_VEL)
        else:
          v = constrain(v, -BURGER_MAX_LIN_VEL, BURGER_MAX_LIN_VEL)
        return v
    def legal_angular_vel(self, w):
        if turtlebot3_model == "burger":
          w = constrain(w, -BURGER_MAX_ANG_VEL, BURGER_MAX_ANG_VEL)
        elif turtlebot3_model == "waffle" or turtlebot3_model == "waffle_pi":
          w = constrain(w, -WAFFLE_MAX_ANG_VEL, WAFFLE_MAX_ANG_VEL)
        else:
          w = constrain(w, -BURGER_MAX_ANG_VEL, BURGER_MAX_ANG_VEL)
        return w
    
    def linear_vel(self, goal_pose, constant=1.5):
        """See video: https://www.youtube.com/watch?v=Qh15Nol5htM."""
        return constant * self.euclidean_distance(goal_pose)

    def steering_angle(self, goal_pose):
        """See video: https://www.youtube.com/watch?v=Qh15Nol5htM."""
        return atan2(goal_pose.y - self.pose.y, goal_pose.x - self.pose.x)

    def angular_vel(self, goal_pose, constant=6):
        """See video: https://www.youtube.com/watch?v=Qh15Nol5htM."""
        return constant * (self.steering_angle(goal_pose) - self.pose.theta)
    
    def P_Controller(self,goal_pose):
        vel_msg=Twist()
        # Linear velocity in the x-axis.
        vel_msg.linear.x = self.linear_vel(goal_pose)
        vel_msg.linear.y = 0
        vel_msg.linear.z = 0

        # Angular velocity in the z-axis.
        vel_msg.angular.x = 0
        vel_msg.angular.y = 0
        vel_msg.angular.z = self.angular_vel(goal_pose)
        return vel_msg
    
    def move2goal(self,goal_x=0,goal_y=0,goal_theta=0,distance_tolerance=0.03,manual_input=False):
        """Moves the turtle to the goal."""
        goal_pose = Pose()
        
        if manual_input:
            # Get the input from the user.
            goal_pose.x = float(input("Set your x goal: "))
            goal_pose.y = float(input("Set your y goal: "))

            # Please, insert a number slightly greater than 0 (e.g. 0.01).
            distance_tolerance = float(input("Set your tolerance: "))
        else:
            goal_pose.x=goal_x
            goal_pose.y=goal_y
            goal_pose.theta=goal_theta
            distance_tolerance=distance_tolerance

        while self.euclidean_distance(goal_pose) >= distance_tolerance:
            if self.pose is None:
                continue # Skip this iterative until the first pose is received.
            
            # Porportional controller.
            # https://en.wikipedia.org/wiki/Proportional_control
            vel_msg=self.P_Controller(goal_pose)
            
            # Proportional controller with maximum speed constraint.
            vel_msg.linear.x=self.legal_linear_vel(vel_msg.linear.x)
            vel_msg.angular.z=self.legal_angular_vel(vel_msg.angular.z)

            # Publishing our vel_msg
            self.velocity_publisher.publish(vel_msg)

            # Publish at the desired rate.
            self.rate.sleep()

        # Stopping our robot after the movement is over.
        vel_msg.linear.x = 0
        vel_msg.angular.z = 0
        self.velocity_publisher.publish(vel_msg)
        print('The target location is reached.')

Now, please open separate bash windows and run the following command to bringup the turtlesim environment first.
```
$ roscore
$ rosrun turtlesim turtlesim_node
```
After running the commands above, you may run the following block to control the turtle in the turtlesim environment.

In [28]:
# Set the goal position as you pleased.
x = TurtleBot()
x.move2goal(goal_x=2,goal_y=3)
x.move2goal(goal_x=0,goal_y=0)

The target location is reached.
The target location is reached.


# What do we want the utility class above for?

* Implement an LQR algorithm that is guarantted to be stable, and can track a path/avoid obstacles rather than simply going to a target location.
* Implement a piecewis